# Install, Import, and Log In

In [10]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist

# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

### Step 0: Install W&B

In [11]:
%%capture
!pip install wandb

### Step 1: Import W&B and Login

In [12]:
import wandb
from wandb.keras import WandbCallback

project_name = 'project-fashion-mnist'
# %env WANDB_API_KEY='1af04e33e3d441eb82eb612e2c001eddec29bccb'

wandb.login(relogin=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Avni Eshchar/.netrc


True

> Side note: If this is your first time using W&B or you are not logged in, the link that appears after running `wandb.login` will take you to sign-up/login page. Signing up is easy!

# Download and Prepare the Dataset


In [13]:
from class_names import class_names

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Subsetting train data and normalizing to [0., 1.]
x_train, x_test = x_train / 255., x_test / 255.

input_shape = (28, 28, 1)
num_classes = len(class_names)

print('Shape of x_train: ', x_train.shape)
print('Shape of y_train: ', y_train.shape)
print('Shape of x_test: ', x_test.shape)
print('Shape of y_test: ', y_test.shape)
print('Num of classes: ', num_classes)

Shape of x_train:  (60000, 28, 28)
Shape of y_train:  (60000,)
Shape of x_test:  (10000, 28, 28)
Shape of y_test:  (10000,)
Num of classes:  10


# Define the Model

Here, we define a standard CNN (with convolution and max-pooling) in Keras.

In [14]:
def Model():
    inputs = keras.layers.Input(shape=input_shape)

    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(inputs)
    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(x)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)

    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(x)
    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(x)
    x = keras.layers.GlobalAveragePooling2D()(x)

    x = keras.layers.Dense(128, activation='relu')(x)
    x = keras.layers.Dense(28, activation='relu')(x)

    outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

    return keras.models.Model(inputs=inputs, outputs=outputs, )

# Train the Model

In [15]:
# Initialize wandb with your project name
run = wandb.init(project=project_name,
                 config={
                     "learning_rate": 0.001,
                     "epochs": 5,
                     "batch_size": 32,
                     "loss_function": "sparse_categorical_crossentropy",
                     "architecture": "CNN",
                     "dataset": "fashion_mnist"
                 })
config = wandb.config  # We'll use this to configure our experiment

# Initialize model like you usually do.
tf.keras.backend.clear_session()
model = Model()
model.summary()

# Compile model like you usually do.
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
model.compile(optimizer, config.loss_function, metrics=['acc'])

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 28)        280       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 28)        7084      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 28)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 28)        7084      
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 28)          7084      
                                                             

In [16]:
# We focus on a subset of images, since this is for human review
val_images, val_labels = x_test[:32], y_test[:32]

_ = model.fit(x_train, y_train,
              epochs=config.epochs, 
              batch_size=config.batch_size,
              validation_data=(x_test, y_test),
              callbacks=[WandbCallback(data_type='image', 
                                       validation_data=(val_images, val_labels), 
                                       labels=class_names)])

run.finish()

Epoch 1/16
2/2 [==============================] - 0s 3ms/step loss: 1.7500 - acc: 0.32


INFO:tensorflow:Assets written to: d:\dev\elbit\ML-course\project-2\wandb\run-20230103_005502-sauar484\files\model-best\assets


INFO:tensorflow:Assets written to: d:\dev\elbit\ML-course\project-2\wandb\run-20230103_005502-sauar484\files\model-best\assets
wandb: Adding directory to artifact (d:\dev\elbit\ML-course\project-2\wandb\run-20230103_005502-sauar484\files\model-best)... Done. 0.0s


40/40 [==============================] - 35s 840ms/step - loss: 1.7500 - acc: 0.3264 - val_loss: 1.0288 - val_acc: 0.6174
Epoch 2/16
2/2 [==============================] - 0s 5ms/step loss: 0.8999 - acc: 0.65


INFO:tensorflow:Assets written to: d:\dev\elbit\ML-course\project-2\wandb\run-20230103_005502-sauar484\files\model-best\assets


...

40/40 [==============================] - 45s 1s/step - loss: 0.4438 - acc: 0.8361 - val_loss: 0.4491 - val_acc: 0.8339
Epoch 16/16
2/2 [==============================] - 0s 8ms/step loss: 0.4312 - acc: 0.83


INFO:tensorflow:Assets written to: d:\dev\elbit\ML-course\project-2\wandb\run-20230103_005502-sauar484\files\model-best\assets


INFO:tensorflow:Assets written to: d:\dev\elbit\ML-course\project-2\wandb\run-20230103_005502-sauar484\files\model-best\assets
wandb: Adding directory to artifact (d:\dev\elbit\ML-course\project-2\wandb\run-20230103_005502-sauar484\files\model-best)... Done. 0.0s


40/40 [==============================] - 45s 1s/step - loss: 0.4312 - acc: 0.8399 - val_loss: 0.4463 - val_acc: 0.8345


acc,▁▅▆▇▇▇▇▇▇███████
epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
loss,█▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁
val_acc,▁▃▄▅▅▆▆▇▇▇▇▇████
val_loss,█▆▅▄▃▃▂▂▂▂▂▂▂▁▁▁
acc,0.8399
best_epoch,15
best_val_loss,0.44629
epoch,15
loss,0.43121
val_acc,0.8345


In [17]:
model.save('./fashion_mnist.hd5')

INFO:tensorflow:Assets written to: ./fashion_mnist.hd5\assets


INFO:tensorflow:Assets written to: ./fashion_mnist.hd5\assets


# Test Hyperparameters with Sweeps

In [9]:
sweep_configuration = {
    'method': 'bayes',
    'name': 'sweep',
    'metric': {
        'goal': 'minimize', 
        'name': 'best_epoch'
    },
    'parameters': {
        'loss_function': {
            'values': ['sparse_categorical_crossentropy'],
            'distribution': 'categorical'
        },
        'epochs': {
            'max': 20,
            'min': 5,
            'distribution': 'int_uniform'
        },
        'learning_rate': {
            'max': 0.006,
            'min': 0.0015,
            'distribution': 'uniform'
        },
        'batch_size': {
            'max': 2048,
            'min': 512,
            'distribution': 'int_uniform'
        }
     }
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project=project_name)
wandb.agent(f'eshcharc/{project_name}/{sweep_id}')

Create sweep with ID: knk4059q
Sweep URL: https://wandb.ai/eshcharc/project-fashion-mnist/sweeps/knk4059q
wandb: Agent Starting Run: wh9qv6kz with config:
	batch_size: 739
	epochs: 15
	learning_rate: 0.005242712833158945
	loss_function: sparse_categorical_crossentropy
wandb: Agent Starting Run: wn5pmbso with config:
	batch_size: 2047
	epochs: 9
	learning_rate: 0.002150239927165999
	loss_function: sparse_categorical_crossentropy
wandb: Agent Starting Run: 7wn69wnw with config:
	batch_size: 589
	epochs: 6
	learning_rate: 0.0024102915295120485
	loss_function: sparse_categorical_crossentropy
wandb: Agent Starting Run: i3ldw49a with config:
	batch_size: 1003
	epochs: 10
	learning_rate: 0.0021102832488502947
	loss_function: sparse_categorical_crossentropy
wandb: Agent Starting Run: sauar484 with config:
	batch_size: 1512
	epochs: 16
	learning_rate: 0.0037793130653754215
	loss_function: sparse_categorical_crossentropy


2023-01-01 21:03:54,132 - wandb.wandb_agent - ERROR - Detected 5 failed runs in a row, shutting down.
